In [12]:
stream = 'http://68.188.109.50/cgi-bin/camera'

In [46]:
import cv2
import matplotlib.pyplot as plt 
from IPython import display
from IPython.display import Image, Video
import numpy as np
import time

In [47]:
net = cv2.dnn.readNet('./net/yolov3.cfg', './net/yolov3.weights')

layer_names = net.getLayerNames()
out_layers_indexes = net.getUnconnectedOutLayers()
out_layers = [layer_names[index[0]-1] for index in out_layers_indexes]

with open('coco.names.txt') as f:
    classes = f.read().split('\n')

In [48]:
parking_area = [
    [0, 730, 180, 1000],
    [180, 740, 280, 1000],
    [280, 750, 350, 1000],
    [350, 770, 450, 1000],
    [450, 820, 800, 1000],
    [800, 840, 1250, 1000],
    [1250, 830, 1580, 1000],
]

In [49]:
def draw_parking_area(img):
    for area in parking_area:
        x1, y1, x2, y2 = area
        start = (x1, y1)
        end = (x2, y2)
        color = (255, 0, 0)
        width = 2
        img = cv2.rectangle(img, start, end, color, width)
    return img

In [50]:
def check_car_in_parking_area(box, class_index):
    name = classes[class_index]
    if name not in ['car', 'truck', 'motorbike', 'bus']:
        return False

    x, y, obj_width, obj_height = box
    car_x_center = x + obj_width // 2
    car_y_center = y + obj_height // 2
    
    for area in parking_area:
        x1, y1, x2, y2 = area
        if x1 <= car_x_center <= x2 and y1 <= car_y_center <= y2:
            return True

    return False

In [51]:
def draw_object(img, index, score, box, selected):
    if check_car_in_parking_area(box, index):
        color = (0, 255, 0)
    else:
        color = (0, 0, 255)

    x,y,w,h = box
    # positioning the rectangle
    start = (x, y) # x, y first point of rectangle
    end = (x + w, y + h) # x, y second point
    width = 2
    img = cv2.rectangle(img, start, end, color, width)
    # positioning the text
    start = (x, y-10)
    font_size = 1
    font = cv2.FONT_HERSHEY_SIMPLEX
    width = 2
    text = classes[index]

    img = cv2.putText(img, text, start, font, font_size, color, width, cv2.LINE_AA)

    return img

In [52]:
count_history = []  # [[10, time.time()], [12, time.time()], [8, time.time()]]
duration = 30  # seconds

def draw_cars_count(img, count):
    now = time.time()
    count_history.append([count, now])
    
    counts = [pair[0] for pair in count_history if pair[1] >= now - 30]
    mid = sum(counts) / len(counts)
    mid_max_counts = [count for count in counts if count >= mid]
    mid_max = sum(mid_max_counts) / len(mid_max_counts)
    mid_max = int(np.ceil(mid_max))

    color = (0, 255, 0)
    start = (50, 150)
    font_size = 3
    font = cv2.FONT_HERSHEY_SIMPLEX 
    width = 3
    text = str(mid_max) + ' cars'
    img = cv2.putText(img, text, start, font, font_size, color, width, cv2.LINE_AA)

    start = (50, 250)
    text = str(45 - mid_max) + ' free parking places'
    img = cv2.putText(img, text, start, font, font_size, color, width, cv2.LINE_AA)
    return img

In [53]:
def apply_yolo(img):
    height, width, depth = img.shape
    blob = cv2.dnn.blobFromImage(img, 1/255, (608, 608), (0,0,0), swapRB=True, crop=False)
    net.setInput(blob)
    outs = net.forward(out_layers)

    class_scores = []
    class_indexes = []
    boxes = []

    for out in outs:
        for obj in out:
            scores = obj[5:]
            class_index = np.argmax(scores)
            class_score = scores[class_index]
            if class_score > 0:
                center_x = int(obj[0] * width)
                center_y = int(obj[1] * height)
                obj_width = int(obj[2] * width)
                obj_height = int(obj[3] * height)

                x = center_x - obj_width // 2
                y = center_y - obj_height // 2

                box = [x, y, obj_width, obj_height]
                boxes.append(box)
                class_indexes.append(class_index)
                class_scores.append(float(class_score))

    chosen_boxes = cv2.dnn.NMSBoxes(boxes, class_scores, 0, 0.4)

    img = draw_parking_area(img)

    selected_count = 0
    for box_index in chosen_boxes:
        box_index = box_index[0]
        box = boxes[box_index]
        class_index = class_indexes[box_index]
        class_score = class_scores[box_index]
        selected = check_car_in_parking_area(box, class_index)
        img = draw_object(img, class_index, class_score, box, selected)

        if selected:
            selected_count+=selected
        img = draw_cars_count(img)

    return img

In [54]:
try:
    cap = cv2.VideoCapture(stream)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = apply_yolo(frame)

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (2560//2, 1440//2))

        plt.figure(figsize=(20,15))
        plt.imshow(frame)
        display.clear_output(wait=True)
        display.display(plt.gcf())

    cap.release()
except KeyboardInterrupt:
    display.clear_output()
    print('Stopped')

TypeError: draw_cars_count() missing 1 required positional argument: 'count'